<a href="https://colab.research.google.com/github/Tpmonkey-Nuttee/IODetector/blob/main/IODetector_for_Thai_Langauge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pythainlp
!pip install stop_words

# Data Preparation

In [ ]:
import pandas as pd

## Spam Dataset

In [ ]:
# Load data from GoogleDrive
# Dataset can be found at: https://transparency.twitter.com/en/reports/information-operations.html
df1 = pd.read_csv("drive/MyDrive/FastAI/thailand_092020_tweets_csv_hashed.csv")

# We only care about tweet itself, nothing else.
df1 = df1[['tweet_text']]
df1.columns = ['text']

# Drop row so it matchs with ham dataset to prevent imbalance dataset.
df1.drop( df1.index[4800:], 0, inplace = True )

# Set the classification value, We will use it as a label.
df1['is_spam'] = 1 


df1.head(-1)

,text,is_spam
0,กกต.จี้“อนาคตใหม่”แจงเงินกู้“ธนาธร” https://t....,1
1,ตายสนิท!'อิศรา'เปิดภ.ง.ด.91ปี61'คุณช่อ'แจ้งราย...,1
2,Let's Get It : ชีวิตติดใบแดง CD กันต์ธีร์ EP....,1
3,อ้าวเฮ้ย ไม่เหมือนที่คุยกันไว้นี่หว่า...อุตส่า...,1
4,เป้นกำลังใจให้นะท่าน,1
...,...,...
4794,RT @army2pr: ข้ามแดนนครพนมปลอดภัย หมอทหารใส่ใจ...,1
4795,"RT @army2pr: ""จะมองผ่านเลนส์..หรือ..มองผ่านตา....",1
4796,RT @army2pr: “มทบ.24 จัดกำลังพลร่วมบริจาคโลหิต...,1
4797,RT @army2pr: “แม่ทัพห่วงใยส่งผู้แทนเยี่ยมตำรวจ...,1


## Ham Dataset
Ham in this case means "Normal Message" (not the ham that you eat)

In [ ]:
# This is a translate version of this spam detection dataset https://www.kaggle.com/benvozza/spam-classification
# I grab only ham messages and translated it using pythainlp translator.
# And save it as a file in Google Drive
df2 = pd.read_csv('drive/MyDrive/FastAI/ham_th.csv')

# Delete unwanted colums, So it matchs with the Spam Dataset
del df2['Unnamed: 0']
del df2['text']
df2.columns = ['text', 'is_spam']

df2.head(-1)

,text,is_spam
0,ไปจนถึงจูรงพ้อยท์ บ้า...มีขายเฉพาะในบูกิสและโล...,0
1,โอเค....ล้อเล่นกับเธอนะ โอนิ...,0
2,เธอไม่บอกเร็วนักเหรอ...เธอเห็นแล้วค่อยบอก...,0
3,ไม่ ฉันไม่คิดว่าเขาไปที่ยูเอสเอฟนะ เขาอาศัยอยู...,0
4,แม้แต่พี่ชายของฉันก็ไม่ชอบคุยกับฉัน พวกเขาปฏิบ...,0
...,...,...
4819,ทําไมคุณไม่รอจนกว่าอย่างน้อยวันพุธเพื่อดูว่าคุ...,0
4820,ฮะ ทําไมละ...,0
4821,เธอจะไปที่ห้างเอสพลานาดจากบ้านไหมอ่ะ,0
4822,"น่าเสียดาย, * อยู่ในอารมณ์นั้น มีอะไรแนะนําอีก...",0


## Final Dataset and Data Preprocessing

In [ ]:
# Merge it together
df = pd.concat([df1, df2])

df.head(-1)

,text,is_spam
0,กกต.จี้“อนาคตใหม่”แจงเงินกู้“ธนาธร” https://t....,1
1,ตายสนิท!'อิศรา'เปิดภ.ง.ด.91ปี61'คุณช่อ'แจ้งราย...,1
2,Let's Get It : ชีวิตติดใบแดง CD กันต์ธีร์ EP....,1
3,อ้าวเฮ้ย ไม่เหมือนที่คุยกันไว้นี่หว่า...อุตส่า...,1
4,เป้นกำลังใจให้นะท่าน,1
...,...,...
4819,ทําไมคุณไม่รอจนกว่าอย่างน้อยวันพุธเพื่อดูว่าคุ...,0
4820,ฮะ ทําไมละ...,0
4821,เธอจะไปที่ห้างเอสพลานาดจากบ้านไหมอ่ะ,0
4822,"น่าเสียดาย, * อยู่ในอารมณ์นั้น มีอะไรแนะนําอีก...",0


In [ ]:
import re
import string

In [ ]:
# possible emoji pattern, We use this to remove all the emoji in tweet text.
emoji_pattern = re.compile("["
  u"\U0001F600-\U0001F64F"  # emoticons
  u"\U0001F300-\U0001F5FF"  # symbols & pictographs
  u"\U0001F680-\U0001F6FF"  # transport & map symbols
  u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
  u"\U00002500-\U00002BEF"  # chinese char
  u"\U00002702-\U000027B0"
  u"\U00002702-\U000027B0"
  u"\U000024C2-\U0001F251"
  u"\U0001f926-\U0001f937"
  u"\U00010000-\U0010ffff"
  u"\u2640-\u2642"
  u"\u2600-\u2B55"
  u"\u200d"
  u"\u23cf"
  u"\u23e9"
  u"\u231a"
  u"\ufe0f"  # dingbats
  u"\u3030"
  "]+", flags=re.UNICODE)

# Replace all emoji with ""
def remove_emoji(string: str): return emoji_pattern.sub(r'', string)

In [ ]:
remove_char = "“”" + string.printable

def clean_text(text: str) -> str:  
    # Remove Hashtag
    text = re.sub(r'#', '', text)

    # Remove twitter tag like @Nuttee
    text = re.sub("/(^|[^@\w])@(\w{1,15})\b/", "", text)

    # Remove emoji
    text = remove_emoji(text)
    
    # Remove some weird character and english alphabet
    return text.translate(str.maketrans('', '', remove_char))

In [ ]:
from pythainlp.tokenize import word_tokenize
from pythainlp.corpus import common
from pythainlp.corpus import wordnet

th_stop = common.thai_stopwords() 

In [ ]:
def split_word(text: str) -> list:
    # tokenize text using pythainlp tokenizer
    tokens = word_tokenize(text)
    
    # remove stop words
    tokens = [i for i in tokens if not i in th_stop]
    
    # Find Stemword in Thai
    tokens_temp = []
    for i in tokens:
        w_syn = wordnet.synsets(i)
        if ( len(w_syn) > 0) and ( len( w_syn[0].lemma_names('tha') ) > 0 ):
            tokens_temp.append( w_syn[0].lemma_names('tha')[0] )
        else:
            tokens_temp.append(i)
    
    tokens = tokens_temp
    
    # Delete blank space
    tokens = [i for i in tokens if not ' ' in i]

    return " ".join(tokens)

In [ ]:
# Clean text
df['text'] = df['text'].apply(clean_text)
# Split it
df['text'] = df['text'].apply(split_word)

df.head(-1)

,text,is_spam
0,กก ต จี้ อนาคต แจง เงินกู้ ธนา ธร,1
1,ตาย สนิท อิศรา ภงด ปี ช่อ แจ้ง รายได้ แสน คืน ...,1
2,ชีวิต ติด ใบแดง กันต์ ธีร ์ ข้อมูล,1
3,อ้าว เฮ้ย เหมือน คุย หว่า อุตส่าห์ รณรงค์ ยกเล...,1
4,เป้ น กำลังใจ ท่าน,1
...,...,...
4819,ทําไม รอ พุธ ดู,0
4820,ฮะ ทําไม,0
4821,ห้าง เอสพลานาด บ้าน ไหม อ่ะ,0
4822,น่าเสียดาย อารมณ์ แนะ ไหม,0


# Creating Model

## Vectorizing

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

# Create a vectorizer
vectorizer = TfidfVectorizer()

# Split our data so that We can use it to train and test.
features_train, features_test, labels_train, labels_test = train_test_split(df['text'], df['is_spam'], test_size=0.3, random_state = 42)

# Make the metrics fit with data first
vectorizer.fit(features_train)

# Transform our data
features_train = vectorizer.transform(features_train)
features_test = vectorizer.transform(features_test)

## GaussianNB

In [ ]:
from sklearn.naive_bayes import GaussianNB

# Create an instance for our model
gnb = GaussianNB()
# Train it with our data
gnb.fit(features_train.toarray(), labels_train)

GaussianNB(priors=None, var_smoothing=1e-09)

In [ ]:
# See how well is it doing
gnb_prediction = gnb.predict(features_test.toarray())
print("Accuracy Score:", accuracy_score(labels_test, gnb_prediction))

print(classification_report(labels_test, gnb_prediction))

Accuracy Score: 0.800207756232687
              precision    recall  f1-score   support

           0       0.73      0.96      0.83      1445
           1       0.94      0.64      0.76      1443

    accuracy                           0.80      2888
   macro avg       0.83      0.80      0.80      2888
weighted avg       0.83      0.80      0.80      2888



## Test it by yourself

In [ ]:
# We can try testing our model here

while True:
  try: s1 = clean_text(input("Input Message: "))  # We will clean it here
  except KeyboardInterrupt: break # Prevent traceback  

  # Transform it first
  s = vectorizer.transform([split_word(s1)])
  # Let see the prediction
  # Except for blank text, We will just ignore that
  is_io = gnb.predict( s.toarray() )[0] == 1 if s1.strip() != "" else False
  
  print("Is IO:", is_io )